In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
hm450_hg19=pd.read_csv('../data/hm450.hg19.manifest.bed', sep='\t', header=None)
hm450_hg19

,0,1,2,3,4
0,chr1,15864,15866,-,cg13869341
1,chr1,18826,18828,-,cg14008030
2,chr1,29406,29408,-,cg12045430
3,chr1,29424,29426,-,cg20826792
4,chr1,29434,29436,-,cg00381604
...,...,...,...,...,...
485572,chrM,6761,6763,-,cg05001044
485573,chrM,6797,6799,+,cg03344490
485574,chrM,7196,7198,-,cg18024856
485575,chrM,9054,9056,+,cg03348902


In [3]:
marker_to_chr_pos_tuple = dict(zip(hm450_hg19[4], zip(hm450_hg19[0], hm450_hg19[2] - 1)))
horvath_coef = pd.read_csv("../data/gb-2013-14-10-r115-S3.csv")
markers = horvath_coef["CpGmarker"]
coefs = horvath_coef["CoefficientTraining"]
horvath_chrs = markers.apply(marker_to_chr_pos_tuple.get)

In [4]:
data = pd.read_csv("../data/sorted_filtered_cytosines_freq.tsv", sep='\t')

In [5]:
chromosome, position = np.transpose([name.split(".") for name in data.values[:, 0]])
position = position.astype(int)

def compare_chr(chr1, p1, chr2, p2):
    return chr1 < chr2 or (chr1 == chr2 and p1 < p2)

def binary_search(c, p):
    left = 0
    right = len(chromosome)
    while right - left > 1:
        mid = (right + left) // 2
        if compare_chr(c, p, chromosome[mid], position[mid]):
            right = mid
        else:
            left = mid
    return left

In [6]:
horvath_ind = horvath_chrs.apply(lambda tup: binary_search(*tup))
data.values[horvath_ind][:, 1:]

array([[90.909, 100.0, 90.0, ..., 95.349, 100.0, 90.323],
       [100.0, 98.571, 100.0, ..., 91.667, 100.0, 100.0],
       [0.0, 0.0, 0.0, ..., 0.87, 1.923, 1.429],
       ...,
       [87.5, 100.0, 37.5, ..., 71.429, 50.0, 100.0],
       [10.526, 0.0, 6.349, ..., 1.389, 3.947, 9.677],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.758]], dtype=object)

In [7]:
import imputation
imputed_missForest = imputation.impute(data.values[horvath_ind][:, 1:].transpose().astype(np.float64) / 100, head=data.columns, method="missForest")
imputed_missForest

        0        1        2        3        4        5        6        7    \
0   0.90909  1.00000  0.00000  0.02597  0.93750  0.11538  0.68132  1.00000   
1   1.00000  0.98571  0.00000  0.16667  0.95385  0.26582  0.66038  1.00000   
2   0.90000  1.00000  0.00000  0.07143  0.78571  0.53659  0.46774  0.50000   
3   1.00000  0.97222  0.00826  0.12500  0.88889  0.42857  0.75610  0.97917   
4   1.00000  0.97619  0.00000  0.15385  1.00000  0.21622  0.70000  0.97297   
5   0.84615  1.00000  0.00000  0.02174  1.00000  0.28571  0.69000  0.96296   
6   0.82353  0.97561  0.00000  0.02632  1.00000  0.43478  0.70536  0.95455   
7   1.00000  1.00000  0.00000  0.03529  1.00000  0.13636  0.85417  1.00000   
8   1.00000  1.00000  0.00000  0.01316  1.00000  0.21429  0.63333  0.79167   
9   0.88462  1.00000  0.00741  0.03409  0.89286  0.34211  0.72115  1.00000   
10  0.73684  1.00000  0.00000  0.36957  1.00000  0.33333  0.78261  0.94737   
11  0.90698  0.93333  0.02190  0.05376  1.00000  0.16667  0.5781

ValueError: Found array with 0 sample(s) (shape=(0, 352)) while a minimum of 1 is required by RandomForestRegressor.